# Part - 2: COVID-19 Time Series Analysis and Prediction using ML.Net framework

## COVID-19
- As per [Wiki](https://en.wikipedia.org/wiki/Coronavirus_disease_2019) **Coronavirus disease 2019** (**COVID-19**) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The disease was first identified in 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.
- The virus had caused a pandemic across the globe and spreading/affecting most of the nations. 
- The purpose of notebook is to visualize the trends of virus spread in various countries and explore features present in ML.Net such as DataFrame.

### Acknowledgement
- [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data) for dataset
- [COVID-19 data visualization](https://www.kaggle.com/akshaysb/covid-19-data-visualization) by Akshay Sb

### Dataset

- [2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE - Time Series](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series).

### Introduction 

This is **Part-2** of our analysis on the COVID-19 dataset provided by Johns Hopkins CSSE. In [**Part-1**](https://github.com/praveenraghuvanshi1512/TechnicalSessions/tree/31052020-virtualmlnet/31052020-virtualmlnet/src/part-1), I did data analysis on the dataset and created some tables and plots for getting insights from it. In Part-2, I'll focus on applying machine learning for making a prediction using time-series API's provided by ML.Net framework. I'll be building a model from scratch on the number of confirmed cases and predicting for the next 7 days. Later on, I'll plot these numbers for better visualization.

[**ML.Net**](https://dotnet.microsoft.com/apps/machinelearning-ai/ml-dotnet) is a cross-platform framework from Microsoft for developing Machine learning models in the .Net ecosystem. It allows .Net developers to solve business problems using machine learning algorithms leveraging their preferred language such as C#/F#. It's highly scalable and used within Microsoft in many of its products such as Bing, Powerpoint, etc.

**Disclaimer**: This is an exercise to explore different features present in ML.Net. The actual and predicted numbers might vary due to several factors such as size and features in a dataset.

### Summary

Below is the summary of steps we'll be performing

1. Define application level items
    - Nuget packages
    - Namespaces
    - Constants
     
2. Utility Functions
    - Formatters    

3. Dataset and Transformations
    - Actual from [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series)
    - Transformed [time_series_covid19_confirmed_global_transposed.csv](time_series_covid19_confirmed_global_transposed.csv)
    
4. Data Classes
    - ConfirmedData : Provides a map between columns in a dataset
    - ConfirmedForecast : Holds predicted values

5. Data Analysis
    - Visualize Data using DataFrame API
    - Display Top 10 Rows - dataframe.Head(10)
    - Display Last 10 Rows - dataframe.Tail(10)
    - Display Dataset Statistics - dataframe.Description()
    - Plot of TotalConfimed cases vs Date

6. Load Data - MLContext
7. ML Pipeline
8. Train Model
9. Evaluate Model
10. Save Model
11. Prediction/Forecasting
12. Prediction Visualization
13. Prediction Analysis
14. Conclusion

**Note** : Graphs/Plots may not render in GitHub due to secutiry reasons, however if you run this notebook locally/binder they will render.

In [1]:
#!about

,.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.115407+d247e88f3f756abad4bd16b47cf8cde187aafabcBuild date: 2020-07-14T23:05:06.5944002Zhttps://github.com/dotnet/interactive


### 1. Define Application wide Items

#### Nuget Packages


In [2]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.TimeSeries"
#r "nuget:Microsoft.Data.Analysis"

// Install XPlot package
#r "nuget:XPlot.Plotly"

Installed package Microsoft.ML version 1.5.1

Installed package Microsoft.Data.Analysis version 0.4.0

Installed package XPlot.Plotly version 3.0.1

Installed package Microsoft.ML.TimeSeries version 1.5.1

#### Namespaces

In [3]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using Microsoft.ML.Transforms.TimeSeries;
using Microsoft.AspNetCore.Html;
using XPlot.Plotly;

#### Constants

In [4]:
const string CONFIRMED_DATASET_FILE = "time_series_covid19_confirmed_global_transposed.csv";
const string MODEL_PATH = "model.zip";
const double SPLIT_RATIO = 0.94;

// Forecast API
const int WINDOW_SIZE = 7;
const int SERIES_LENGTH = 30;
const int HORIZON = 7;
const float CONFIDENCE_LEVEL = 0.95f;

// Dataset
const int DEFAULT_ROW_COUNT = 10;
const string TOTAL_CONFIRMED_COLUMN = "TotalConfirmed";
const string DATE_COLUMN = "Date";

### 2. Utility Functions

#### Formatters

By default the output of DataFrame is not proper and in order to display it as a table, we need to have a custom formatter implemented as shown in next cell. 

In [5]:
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = DEFAULT_ROW_COUNT;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");

### 3. Dataset and Transformations

#### Download Dataset
- Actual Dataset: [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series)
- Transformed Dataset: [time_series_covid19_confirmed_global_transposed.csv](time_series_covid19_confirmed_global_transposed.csv)


I'll be using COVID-19 time series dataset from [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series) and will be performing predictions using **time_series_covid19_confirmed_global.csv** file.

The data present in these files have name of the countries as Rows and dates as columns which makes it difficult to map to our classes while loading data from csv. Also, it contains data per country wise. In order to keep things simple I'll work with global count of COVID-19 cases and not specific country.

I have done few transformations to the dataset as below and created transformed csv's
- Sum cases from all the countries for a specific date
- Just have two rows with Date and Total 
- Applied transformation to the csv for converting Rows into Columns and vice-versa. [Refer](https://support.office.com/en-us/article/transpose-rotate-data-from-rows-to-columns-or-vice-versa-3419f2e3-beab-4318-aae5-d0f862209744) for transformation.
- Below transposed files have been saved in the current github directory. There is no change in dataset. The files have data till 05-27-2020
    - [time_series_covid19_confirmed_global_transposed.csv](time_series_covid19_confirmed_global_transposed.csv) : Columns - **Date, TotalConfirmed**

##### Before transformation

<img src=".\assets\time-series-before-transformation.png" alt="Time Series data before transofmation" style="zoom: 80%;" />

#### After transformation

<img src=".\assets\time-series-after-transformation.png" alt="Time Series data after transofmation" style="zoom: 80%;" />

### 4. Data Classes

Now, we need to create few data structures to map to columns within our dataset.

#### Confirmed cases

In [6]:
/// <summary>
/// Represent data for confirmed cases with a mapping to columns in a dataset
/// </summary>
public class ConfirmedData
{
    /// <summary>
    /// Date of confirmed case
    /// </summary>
    [LoadColumn(0)]
    public DateTime Date;

    /// <summary>
    /// Total no of confirmed cases on a particular date
    /// </summary>
    [LoadColumn(1)]
    public float TotalConfirmed;
}

In [7]:
/// <summary>
/// Prediction/Forecast for Confirmed cases
/// </summary>
internal class ConfirmedForecast
{
    /// <summary>
    /// No of predicted confirmed cases for multiple days
    /// </summary>
    public float[] Forecast { get; set; }
    
    /// <summary>
    /// The predicted minimum values for the forecasted period.
    /// </summary>
    public float[] LowerBoundConfirmed { get; set; }

    /// <summary>
    /// The predicted maximum values for the forecasted period.
    /// </summary>
    public float[] UpperBoundConfirmed { get; set; }
}

In [8]:
/// <summary>
/// Represents the output to be used for display
/// </summary>
public class ForecastOutput
{
    /// <summary>
    /// Date of confirmed case
    /// </summary>
    public DateTime Date { get; set; }

    /// <summary>
    /// Number of actual confirmed cases
    /// </summary>
    public float ActualConfirmed { get; set; }

    /// <summary>
    /// Lower bound confirmed cases
    /// </summary>
    public float LowerEstimate { get; set; }

    /// <summary>
    /// Predicted confirmed cases
    /// </summary>
    public float Forecast { get; set; }

    /// <summary>
    /// Upper bound confirmed cases
    /// </summary>
    public float UpperEstimate { get; set; }
}

### 5. Data Analysis

For loading data from csv, first we need to create MLContext that acts as a starting point for creating a machine learning model in ML.Net. Few things to note
- Set hasHeader as true as our dataset has header
- Add separatorChar to ',' as its a csv

#### Visualize Data - DataFrame

In [9]:
var predictedDf = DataFrame.LoadCsv(CONFIRMED_DATASET_FILE);

In [10]:
predictedDf.Head(DEFAULT_ROW_COUNT)

0,1/22/2020,555
1,1/23/2020,654
2,1/24/2020,941
3,1/25/2020,1434
4,1/26/2020,2118
5,1/27/2020,2927
6,1/28/2020,5578
7,1/29/2020,6166
8,1/30/2020,8234
9,1/31/2020,9927


In [11]:
predictedDf.Tail(DEFAULT_ROW_COUNT)

0,4/28/2020,3097229
1,4/29/2020,3172287
2,4/30/2020,3256910
3,5/1/2020,3343777
4,5/2/2020,3427584
5,5/3/2020,3506729
6,5/4/2020,3583055
7,5/5/2020,3662691
8,5/6/2020,3755341
9,5/7/2020,3845718


In [12]:
predictedDf.Description()

0,Length (excluding null values),107
1,Max,3845718
2,Min,555
3,Mean,923109.56


##### Number of Confirmed cases over Time

In [13]:
// Number of confirmed cases over time
var totalConfirmedDateColumn = predictedDf.Columns[DATE_COLUMN];
var totalConfirmedColumn = predictedDf.Columns[TOTAL_CONFIRMED_COLUMN];

var dates = new List<DateTime>();
var totalConfirmedCases = new List<string>();
for (int index = 0; index < totalConfirmedDateColumn.Length; index++)
{
    dates.Add(Convert.ToDateTime(totalConfirmedDateColumn[index]));
    totalConfirmedCases.Add(totalConfirmedColumn[index].ToString());
}

In [14]:
var title = "Number of Confirmed Cases over Time";
var confirmedTimeGraph = new Graph.Scattergl()
    {
        x = dates.ToArray(),
        y = totalConfirmedCases.ToArray(),
        mode = "lines+markers"
    };

var chart = Chart.Plot(confirmedTimeGraph);
chart.WithTitle(title);
display(chart);

**Analysis**
- Duration: 1/22/2020 through 5/7/2020
- Total records: 107
- Case on first day: 555
- Case on last day: 3.85 Million
- No of confirmed cases was low in the beginning, there was first jump around 2/12/2020 and an exponential jump around 3/22/2020.
- Cases have been increasing at an alarming rate in the past two months.

### 6. Load Data - MLContext

In [15]:
var context = new MLContext();
var data = context.Data.LoadFromTextFile<ConfirmedData>(CONFIRMED_DATASET_FILE, hasHeader: true, separatorChar: ',');

#### Split Dataset

In [16]:
var totalRows = (int)data.GetColumn<float>("TotalConfirmed").ToList().Count;
int numTrain = (int)(SPLIT_RATIO * totalRows);
var confirmedAtSplit = (int)data.GetColumn<float>("TotalConfirmed").ElementAt(numTrain);
var startingDate = data.GetColumn<DateTime>("Date").FirstOrDefault();
var endDate = data.GetColumn<DateTime>("Date").LastOrDefault();
var dateAtSplit = data.GetColumn<DateTime>("Date").ElementAt(numTrain);
    
IDataView trainData = context.Data.FilterRowsByColumn(data, "TotalConfirmed", upperBound: confirmedAtSplit);
IDataView testData = context.Data.FilterRowsByColumn(data, "TotalConfirmed", lowerBound: confirmedAtSplit);

display(h4($"Training dataset range : {startingDate.ToShortDateString()} to {dateAtSplit.ToShortDateString()}"));
display(h4($"Test dataset range : {dateAtSplit.AddDays(1).ToShortDateString()} to {endDate.ToShortDateString()}"));

Training dataset range : 1/22/2020 to 5/1/2020

Test dataset range : 5/2/2020 to 5/7/2020

### 7. ML Pipeline

For creating ML Pipeline for a time-series analysis, we'll use [Single Spectrum Analysis](https://en.wikipedia.org/wiki/Singular_spectrum_analysis). ML.Net provides built in API for same, more details could be found at [TimeSeriesCatalog.ForecastBySsa](https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.timeseriescatalog.forecastbyssa?view=ml-dotnet) 

In [17]:
// Parameters

display($"No of Training samples: {numTrain}");
display($"Series Lenght: {SERIES_LENGTH}");
display($"Window size: {WINDOW_SIZE}");
display($"Forecast perion(Days): {HORIZON}");
display($"CONFIDENCE: {CONFIDENCE_LEVEL}");

No of Training samples: 100

Series Lenght: 30

Window size: 7

Forecast perion(Days): 7

CONFIDENCE: 0.95

In [18]:
var pipeline = context.Forecasting.ForecastBySsa(
                nameof(ConfirmedForecast.Forecast),
                nameof(ConfirmedData.TotalConfirmed),
                WINDOW_SIZE,
                SERIES_LENGTH,
                trainSize: numTrain,
                horizon: HORIZON,
                confidenceLevel: CONFIDENCE_LEVEL,
                confidenceLowerBoundColumn: nameof(ConfirmedForecast.LowerBoundConfirmed),
                confidenceUpperBoundColumn: nameof(ConfirmedForecast.UpperBoundConfirmed));

### 8. Train Model

We are ready with our pipeline and ready to train the model

In [19]:
var model = pipeline.Fit(data);

### 9. Evaluate Model

We have trained our model and its ready for evaluation. We'll evaluate our model on test dataset and see how it has performed by determining various matrices such as MAE and RMSE

In [20]:
IDataView predictions = model.Transform(testData);

IEnumerable<float> actual =
    context.Data.CreateEnumerable<ConfirmedData>(testData, true)
        .Select(observed => observed.TotalConfirmed);

IEnumerable<float> forecast =
    context.Data.CreateEnumerable<ConfirmedForecast>(predictions, true)
        .Select(prediction => prediction.Forecast[0]);

var metrics = actual.Zip(forecast, (actualValue, forecastValue) => actualValue - forecastValue);

var MAE = metrics.Average(error => Math.Abs(error)); // Mean Absolute Error
var RMSE = Math.Sqrt(metrics.Average(error => Math.Pow(error, 2))); // Root Mean Squared Error

display("Evaluation Metrics");
display("---------------------");
display($"Mean Absolute Error: {MAE:F3}");
display($"Root Mean Squared Error: {RMSE:F3}\n");

Evaluation Metrics

---------------------

Mean Absolute Error: 85687.070

Root Mean Squared Error: 480481.195


### 10. Save Model

Once we are confirm on the model performance, we can save it as a zip file which can be used later for making prediction.

In [21]:
var forecastingEngine = model.CreateTimeSeriesEngine<ConfirmedData, ConfirmedForecast>(context);
forecastingEngine.CheckPoint(context, MODEL_PATH);

### 11. Prediction/Forecasting - 7 days

Our model is trained and we need to do prediction for next 7(Horizon) days.
Time-series provides its own engine for making prediction which is similar to PredictionEngine present in ML.Net. Predicted values show an increasing trend which is in alignment with recent past values.

In [22]:
var forecasts = forecastingEngine.Predict();

In [23]:
var forecastOutputs = context.Data.CreateEnumerable<ConfirmedData>(testData, reuseRowObject: false)
                .Take(HORIZON)
                .Select((ConfirmedData confirmedData, int index) =>
                {
                    float lowerEstimate = Math.Max(0, forecasts.LowerBoundConfirmed[index]);
                    float estimate = forecasts.Forecast[index];
                    float upperEstimate = forecasts.UpperBoundConfirmed[index];

                    return new ForecastOutput
                    {
                        ActualConfirmed = confirmedData.TotalConfirmed,
                        Date = confirmedData.Date,
                        Forecast = estimate,
                        LowerEstimate = lowerEstimate,
                        UpperEstimate = upperEstimate
                    };
                });

PrimitiveDataFrameColumn<DateTime> forecastDates = new PrimitiveDataFrameColumn<DateTime>("Date"); // Default length is 0.
PrimitiveDataFrameColumn<float> actualConfirmedCases = new PrimitiveDataFrameColumn<float>("ActualConfirmed"); // Makes a column of length 3. Filled with nulls initially
PrimitiveDataFrameColumn<float> forecastCases = new PrimitiveDataFrameColumn<float>("Forecast"); // Makes a column of length 3. Filled with nulls initially
PrimitiveDataFrameColumn<float> lowerEstimates = new PrimitiveDataFrameColumn<float>("LowerEstimate"); // Makes a column of length 3. Filled with nulls initially
PrimitiveDataFrameColumn<float> upperEstimates = new PrimitiveDataFrameColumn<float>("UpperEstimate"); // Makes a column of length 3. Filled with nulls initially

foreach (var output in forecastOutputs)
{
    forecastDates.Append(output.Date);
    actualConfirmedCases.Append(output.ActualConfirmed);
    forecastCases.Append(output.Forecast);
    lowerEstimates.Append(output.LowerEstimate);
    upperEstimates.Append(output.UpperEstimate);
}

display(h4("Total Confirmed Cases Forecast"));
var forecastDataFrame = new DataFrame(forecastDates, actualConfirmedCases, lowerEstimates, forecastCases, upperEstimates);
display(forecastDataFrame)

Total Confirmed Cases Forecast

0,2020-05-01 00:00:00Z,3343777,3918628.8,3931320.5,3944012.2
1,2020-05-02 00:00:00Z,3427584,3936862.2,4007682,4078501.8
2,2020-05-03 00:00:00Z,3506729,3962438,4085376,4208314
3,2020-05-04 00:00:00Z,3583055,3990854.8,4170695.5,4350536
4,2020-05-05 00:00:00Z,3662691,4018113,4263132.5,4508152
5,2020-05-06 00:00:00Z,3755341,4026267.8,4352279,4678290.5
6,2020-05-07 00:00:00Z,3845718,4007697.2,4433297,4858896.5


### 12. Prediction Visualization

In [24]:
var predictionStartDate = dateAtSplit.AddDays(-1); // lastDate.AddDays(1);

var newDates = new List<DateTime>();
var fullDates = new List<DateTime>();
fullDates.AddRange(dates.Take(numTrain));

var fullTotalConfirmedCases = new List<string>();
fullTotalConfirmedCases.AddRange(totalConfirmedCases.Take(numTrain));

for (int index = 0; index < HORIZON; index++)
{
    var nextDate = predictionStartDate.AddDays(index + 1);
    newDates.Add(nextDate);
    fullTotalConfirmedCases.Add(forecasts.Forecast[index].ToString());
}

fullDates.AddRange(newDates);

var layout = new Layout.Layout();
layout.shapes = new List<Graph.Shape>
{
    new Graph.Shape
    {
        x0 = predictionStartDate,
        x1 = predictionStartDate,
        y0 = "0",
        y1 = "1",
        xref = 'x',
        yref = "paper",
        line = new Graph.Line() {color = "red", width = 2}
    }
};

var predictionChart = Chart.Plot(
    new[]
    {
        new Graph.Scattergl()
        {
            x = fullDates.ToArray(),
            y = fullTotalConfirmedCases.ToArray(),
            mode = "lines+markers"
        }
    },
    layout
);

predictionChart.WithTitle("Number of Confirmed Cases over Time");
display(predictionChart);

Graph.Scattergl[] scatters = {
    new Graph.Scattergl() {
        x = newDates,
        y = forecasts.UpperBoundConfirmed,
        fill = "tonexty",
        name = "Upper bound"
    },
    new Graph.Scattergl() {
        x = newDates,
        y = forecasts.Forecast,
        fill = "tonexty",
        name = "Forecast"
    },
    new Graph.Scattergl() {
        x = newDates,
        y = forecasts.LowerBoundConfirmed,
        fill = "tonexty",
        name = "Lower bound"
    }
};

var predictionChart2 = Chart.Plot(scatters);
predictionChart2.Width = 600;
predictionChart2.Height = 600;
display(predictionChart2);

### 13. Analysis

Looking at the first plot above, it seems our ML model has performed reasonably well. The red line represents the data on future dates(7 days) taken from test data between May 5 through May 11. Looking at the plot, there is a sudden rise at the beginning which could be accounted due to insufficient data as we have only 107 records. However we see an increasing trend for next 7 days in alignment with previous confirmed cases. We can extend this model for predicting confirmed cases for any number of days by changing HORIZON constant value. This plot is helpful in analysing the increased number of cases and allow authorities to take precautionary measures to keep the numbers low.

## Conclusion

I hope you have enjoyed reading the notebook, and might have got some idea on the powerful framework ML.Net. ML.Net is a very fast emerging framework for .Net developers which abstracts lot of complexity present in the field of Data science and Machine Learning. The focus of Part-2 notebook is leverage ML.Net for making predictions using time-series API. The model generated can be saved as a zip file and used in different applications.

Feedback/Suggestion/Correction is welcome. Please reach out to me through below channels

**Contact**

**Twitter  :** @praveenraghuvan

**Email    :** praveenraghuvanshi@gmail.com 

**LinkedIn :** https://in.linkedin.com/in/praveenraghuvanshi  
**Github   :** https://github.com/praveenraghuvanshi

**dev.to   :** https://dev.to/praveenraghuvanshi



## References
- [Tutorial: Forecast bike rental service demand with time series analysis and ML.NET](https://docs.microsoft.com/en-us/dotnet/machine-learning/tutorials/time-series-demand-forecasting#evaluate-the-model)
- [Time Series Forecasting in ML.NET and Azure ML notebooks](https://github.com/gvashishtha/time-series-mlnet/blob/master/time-series-forecast.ipynb)

#  ******************** Be Safe **********************